# **Pytorch Project Template**
- 개발 초기에는 대화식 개발 과정이 유리 → 학습과 디버깅 등 지속적인 확인
- 배포 및 공유 단계에서는 notebook 공유가 어려움 → 쉬운 재현의 어려움, 실행순서 꼬임
- DL 코드도 하나의 프로그램: 개발 용이성 확보와 유지보수 향상 필요
- 프로젝트= OOP+모듈
- 가장 좋은 Pytorch 템플릿
  https://github.com/victoresque/pytorch-template

### 모듈 구성
1. 실행
2. 설정
3. base abstract module - 만들려고 하는 모델 아키텍쳐 설정
4. data - 데이터 저장
5. model - architecture, loss 설정 등
6. 저장소 -로그, 모델 상태
7. 학습 수행
8. 로깅 설정 - logger
9. 유틸리티

In [1]:
!pip3 install torch
!pip3 install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/victoresque/pytorch-template

Cloning into 'pytorch-template'...
remote: Enumerating objects: 1516, done.
remote: Total 1516 (delta 0), reused 0 (delta 0), pack-reused 1516
Receiving objects: 100% (1516/1516), 288.08 KiB | 16.95 MiB/s, done.
Resolving deltas: 100% (848/848), done.


In [4]:
!ls

drive  pytorch-template  sample_data


In [5]:
%cd /content/pytorch-template

/content/pytorch-template


In [6]:
!python new_project.py MNIST-example

New project initialized at /content/pytorch-template/MNIST-example


In [7]:
# colab 환경에 원격 접속
NGROK_TOKEN = '2MwpVIqLNoeTWZVygeefvFO8NNA_2dXLEmijgip1ozAG9M93R' # ngrok 토큰
PASSWORD = 'upstage' # 비밀번호 설정

In [8]:
# 노트북에서 ssh로 colab 환경에 접속하게 해줌
!pip install colab-ssh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
#ssh에 접근할 수 있게 서버가 돌아감
from colab_ssh import launch_ssh
launch_ssh(NGROK_TOKEN, PASSWORD)

we highly recommend that update your code by following this documentation https://github.com/WassimBenzarti/colab-ssh#getting-started
Successfully running 4.tcp.ngrok.io:13638
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

  Host google_colab_ssh
    HostName 4.tcp.ngrok.io
    User root
    Port 13638
    


이후 vscode 접속
1. ctrl+shift+p -> remote add new host
2. ssh root@8.tcp.ngrok.io -p 10370
3. remote connect host
4. (접속 잘 안되면 ssh setting 들어가서 absolute 경로 C:\Users\yoonpyo\.ssh\config으로 바꾸기)


##  Pytorch project 템플릿 구성
- train.py: 파이썬 코드 수행시켜주는 주체, parse_config.py의 from_args 함수에서 객체를 반환받아 main 함수 실행
- parse_config.py: from_args에서 util.py의 read_json 함수에서 json파일을 가져와 Ordereddict 형식으로 저장해줌. 이것을 config에 저장
- data_loaders.py: config.json에서 모델을 가져오는 파일
- config.json: data_loaders에 있는 다양한 모델들의 학습 설정 저장
- model.py:추후에 다른 모델을 추가하고 싶을 때 여기에 추가, 모델을 추가해주고 config.json에서 수정만 해주면 됨
- trainer.py:학습이 이뤄지는 공간. data_loader.py에서 모델 가져옴, 그레디언트 계산
  - base_trainer.py : Loggin에 대한 코드가 저장되어 있고, Epoch가 어느정도 진행되면 자동으로 학습 모델을 중간 저장하는 코드가 입력되어 있다

### parse_config.py의 init_obj()

    def init_obj(self, name, module, *args, **kwargs):
        """
        Finds a function handle with the name given as 'type' in config, and returns the
        instance initialized with corresponding arguments given.

        `object = config.init_obj('name', module, a, b=1)`
        is equivalent to
        `object = module.name(a, b=1)`
        """
        module_name = self[name]['type'] # ⏰ config.json의 dataloader의 type
        module_args = dict(self[name]['args']) # ⏰ config.json의 dataloader의 type의 args
        assert all([k not in module_args for k in kwargs]), 'Overwriting kwargs given in config file is not allowed'
        module_args.update(kwargs)
        return getattr(module, module_name)(*args, **module_args)
        # ⏰ getattr를 쓰는 이유: 매번 하드코딩으로 모델을 수정하지 않고 config.json파일만 바꿔서 쓰기위해


### __getitem(): 인덱스 값을 넣어주면 값을 출력해줌
    def __getitem__(self, name):
        """Access items like ordinary dict."""
        return self.config[name]

In [11]:
class Test(object):
    def __getitem__(self, items):
        print (type(items), items)

# Driver code
test = Test()
test[5]
test[5:65:5]
test['GeeksforGeeks']
test[1, 'x', 10.0]
test['a':'z':2]
test[object()]

<class 'int'> 5
<class 'slice'> slice(5, 65, 5)
<class 'str'> GeeksforGeeks
<class 'tuple'> (1, 'x', 10.0)
<class 'slice'> slice('a', 'z', 2)
<class 'object'> <object object at 0x7f5328570d50>
